In [4]:
import zarr
import xarray as xr
import numpy as np
from boto3 import client as boto_client

In [5]:

def CRSaccess(fname, s3bucket=False, Verb=False):
    """
    Access the CRS file
    Return CRS filename with path (absolute path) for "local" access
    Return CRS data as object for "cloud access"
    Either way, the return value can be open by Xarray as netcdf file object
    """

    print("\%% Accessing data from Cloud. This may take a little time...\n")
    s3 = boto_client('s3')
    fileobj = s3.get_object(Bucket=s3bucket, Key=fname)
    fileCRS = fileobj['Body'].read()

    return fileCRS

def add24hr(hr):
    """Correction of time in CRS for going over the next day in UTC"""
    b = np.where(hr < hr[0])
    hr[b] = hr[b] + 24
    return hr

In [6]:
folder = f"/tmp/crs_olympex/zarr/11111111"
bucket_name="ghrc-fcx-field-campaigns-szg"
s3_raw_file_key= "Olympex/instrument-raw-data/crs/olympex_CRS_20151110_172815-20151110_175946_2_v01a.nc"
file = s3_raw_file_key

date = file.split("_")[2]
base_time = np.datetime64('{}-{}-{}'.format(date[:4], date[4:6], date[6:]))

In [7]:
# read from s3 url (file) in s3 bucket.
fileObj = CRSaccess(file, s3bucket=bucket_name)

# open dataset.
with xr.open_dataset(fileObj, decode_cf=False) as ds:
    #### time correction start
    # !!! (logic????) added for time correction for time (hour) over 24h UTC
    hr = add24hr(ds['timed'].values)

    # !!! addition of date to all time rows i.e. {date + time} for all rows.
    # Note: "delta" is an intermediatery before final time correction.
    delta = (hr * 3600).astype('timedelta64[s]') + base_time
    # now delta will have this sort of value: ['2015-11-10T17:28:16' '2015-11-10T17:28:16' '2015-11-10T17:28:17' ...
    # '2015-11-10T18:01:05' '2015-11-10T18:01:05' '2015-11-10T18:01:05']
    
    #### time correction end
    
    # Data COLS EXTRACT 
    ref = ds["zku"].values #CRS radar reflectivity
    rad_range = ds["range"].values

    lat = ds['lat'].values
    lon = ds['lon'].values
    alt = ds['altitude'].values # altitude of aircraft in meters
    roll = ds["roll"].values
    pitch = ds["pitch"].values
    head = ds["head"].values
num_col = ref.shape[0] # number of cols, say 7903
num_row = ref.shape[1] # number of rows, say 757

\%% Accessing data from Cloud. This may take a little time...



In [13]:
ref[np.isfinite(ref)].max()


60.44812

In [14]:
ref[np.isfinite(ref)].min()


-1103.8334